# Generate atlas and metadata that can be by Facets Dive

This Notebook serves as a simple example for how to generate an atlas that can be visualized using Facets Dive. 

In [ ]:
!pip install scikit-image

In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from skimage import io

## Download the Fashion-MNIST dataset

In [ ]:
!git clone https://github.com/zalandoresearch/fashion-mnist.git fmnist

In [ ]:
import sys
sys.path.append('fmnist/utils')
from mnist_reader import load_mnist

X_train, y_train = load_mnist('fmnist/data/fashion', kind='train')
X_test, y_test = load_mnist('fmnist/data/fashion', kind='t10k')

In [ ]:
X_train = X_train[0:400]
y_train = y_train[0:400]

In [ ]:
label_names = {
    0: 't-shirt/top',
    1: 'trouser',
    2: 'pullover',
    3: 'dress',
    4: 'coat',
    5: 'sandal',
    6: 'shirt',
    7: 'sneaker',
    8: 'bag',
    9: 'angle_boot'
}

## Generate some artificial meta-data for each image

In [ ]:
# store the indices of each image as IDs
im_ids = range(X_train.shape[0])

# get the brightness of each image
im_brightness = np.mean(X_train, axis=1)

# get the number of non-zero pixels per image
non_zeros = np.sum(X_train > 0, axis=1)

class_name = [label_names[y] for y in y_train]

# output to JSON file

df = pd.DataFrame({'id': im_ids, 'class_id': y_train, 'class_name': class_name, 'brightness': im_brightness, 'non-zeros': non_zeros})
jsonstr = df.to_json(orient='records')

with open('fmnist_data.json', 'w') as f:
    f.write(jsonstr)

In [ ]:
df.to_csv('fmnist.csv', index=None)


## Create sprite atlas

We use Bazel to generate a "sprite atlas", which is a mosaic of all the small images.

Since we already have all the images in a Numpy array, we can generate our own sprite atlas.

### Manually create sprite atlas (i.e. mosaic)

In [ ]:
# reshape image data from rows into square images
im_w = im_h = 28
num_images = X_train.shape[0]
X = np.reshape(X_train, (num_images, im_h, im_w))

# calculate how many rows and columns we need (round up)
cols = int(np.ceil(np.sqrt(X_train.shape[0])))
rows = int(np.ceil(X_train.shape[0] * 1. / cols))

# copy image data into mosaic
mosaic = np.zeros((rows * im_h, cols * im_w), dtype=np.uint8)

for row in range(rows):
    for col in range(cols):
        im_index = col * rows + row
        if im_index < num_images:
            mosaic[col * im_h:(col+1) * im_h, row * im_w:(row+1) * im_w] = X[im_index, :, :]

In [ ]:
# write mosaic image to disk
io.imsave('sprite_atlas.png', mosaic)

In [ ]:
jj = 41
plt.imshow(X[jj, :, :])
print(label_names[y_train[jj]])